In [1]:
import numpy as np
from functions import sigmoid,softmax,dsigmoid
from util import spiral_data_gen
from sklearn.metrics import classification_report
from sklearn.datasets import *

In [4]:
#X,y=spiral_data_gen(False)
X,y=load_wine()['data'],load_wine()['target']


X=X.T
D,N=X.shape
print(D,N)
K=len(np.unique(y))

hidden_size=50
W0 = 0.01 * np.random.randn(hidden_size,D)
b0 = np.zeros((hidden_size,1))
W1 = 0.01 * np.random.randn(hidden_size,hidden_size)
b1 = np.zeros((hidden_size,1))
W2 = 0.01 * np.random.randn(K,hidden_size)
b2 = np.zeros((K,1))
    
num_epoch=100_000
mode=num_epoch//10
step_size=.001

for epoch in range(num_epoch):
    
        # forward pass
        S0 = W0.dot(X)+ b0
        Z0 = np.maximum(0, S0)
        
        S1 = W1.dot(Z0)+ b1
        Z1 = np.maximum(0, S1)
        
        S2 = W2.dot(Z1) + b2
        Z2  = softmax(S2)

        # compute the loss
        corect_logprobs = -np.log(Z2[y,range(N)])
        loss = np.sum(corect_logprobs)/N
        
        if epoch % mode == 0:
            print("Epoch %d: loss %f" % (epoch, loss))

        # backward pass

        # Derivative of loss w.r.t. S2 the input of of softmax.
        dS2 = Z2
        dS2[y,range(N)] -= 1
        dS2 /= N

        # ScoreGate => Multiply and Addition gate
        db2 = np.sum(dS2, axis=1, keepdims=True)
        dW2 = dS2.dot(Z1.T)
        dZ1  = W2.T.dot(dS2) 

        # ReluGate
        dZ1[S1 <= 0] = 0
        dS1=dZ1
        
        # ScoreGate
        dW1 = dS1.dot(Z0.T)
        db1 = np.sum(dS1, axis=1, keepdims=True)
        dZ0  = W1.T.dot(dS1)

        # ReluGate
        dZ0[S0 <= 0] = 0
        
        # ScoreGate
        dS0=dZ0
        dW0=dS0.dot(X.T)
        db0 = np.sum(dS0, axis=1, keepdims=True)
        #dX=W0.T.dot(dZ0)



        # perform a parameter update
        W0 += -step_size * dW0
        b0 += -step_size * db0
        
        W1 += -step_size * dW1
        b1 += -step_size * db1
        
        W2 += -step_size * dW2
        b2 += -step_size * db2
    
# forward pass
S0 = W0.dot(X)+ b0
Z0 = np.maximum(0, S0)

S1 = W1.dot(Z0)+ b1
Z1 = np.maximum(0, S1)

S2 = W2.dot(Z1) + b2
Z2  = softmax(S2)
predicted_class = np.argmax(Z2, axis=0)

print(classification_report(y, predicted_class))

13 178
Epoch 0: loss 1.092955
Epoch 10000: loss 0.578790
Epoch 20000: loss 0.474608
Epoch 30000: loss 0.411107
Epoch 40000: loss 0.418407
Epoch 50000: loss 0.446248
Epoch 60000: loss 0.309231
Epoch 70000: loss 0.377409
Epoch 80000: loss 0.477124
Epoch 90000: loss 0.449588
              precision    recall  f1-score   support

           0       0.96      0.88      0.92        59
           1       0.89      0.94      0.92        71
           2       0.96      0.98      0.97        48

    accuracy                           0.93       178
   macro avg       0.94      0.93      0.94       178
weighted avg       0.93      0.93      0.93       178



In [8]:
class MatMulGate:
    def __init__(self):
        pass
    def forward(self,W,b,X):
        self.W=W
        self.X=X
        return self.W.dot(self.X)+b
    def backward(self,dL):
        dW=dL.dot(self.X.T)
        dX=self.W.T.dot(dL)
        db=np.sum(dL, axis=1, keepdims=True)
        return dW,dX,db
class ReluGate:
    
    def __init__(self):
        pass
    
    def forward(self,X):
        self.X=X
        return np.maximum(0, self.X)

    def backward(self,dL):
        dL[self.X <= 0] = 0
        return dL
    
class SoftmaxGate:
    def __init__(self):
        pass
    def forward(self,X):
        self.Z=softmax(X)
        return self.Z
    def backward(self,dL):
        return dL
    
class ComputationalGraph:
    def __init__(self):
        self.gates=[]
        
    def add(self,shape,gate):
        fout,fin=shape
        W = 0.01 * np.random.randn(fout,fin)
        b = np.zeros((fout,1))
        
        self.gates.append(((W,b), MatMulGate(),gate))
        
    def forward(self,X):
        for t in self.gates:
            (W,b),score,sigma=t
            S=score.forward(W,b,X)
            X=sigma.forward(S)
        self.Z=X
        return self.Z
    
    def backward(self,y):
        
        # Derivative of loss w.r.t. S2 the input of of softmax.
        dL = self.Z
        dL[y,range(N)] -= 1
        for t in reversed(self.gates):
            (W,b),score,sigma=t
            dW,dL,db=score.backward(sigma.backward(dL))
            W+=-.001*dW
            b+=-.001*db
            

In [10]:
X,y=spiral_data_gen(False)
X=X.T
D,N=X.shape
K=len(np.unique(y))

hidden_size=50

model=ComputationalGraph()
model.add((hidden_size,D),ReluGate())
model.add((hidden_size,hidden_size),ReluGate())
model.add((K,hidden_size),SoftmaxGate())
num_epoch=10_000
mode=num_epoch//10


for epoch in range(num_epoch):
    preds=model.forward(X)
    
    if epoch%mode==0:
        loss = (-np.log(preds[y,range(N)])).mean()# mean corect_logprobs
        print('{0}.th epoch Loss:{1}'.format(epoch,loss))
    model.backward(y)
    
    
y_head = np.argmax(model.forward(X), axis=0)
print(classification_report(y, y_head))

0.th epoch Loss:1.0986098813186431
1000.th epoch Loss:0.5582987428439258
2000.th epoch Loss:0.041315717949864474
3000.th epoch Loss:0.02692384151150519
4000.th epoch Loss:0.025109393590978225
5000.th epoch Loss:0.02032171608476702
6000.th epoch Loss:0.019059972354041003
7000.th epoch Loss:0.01786430545616846
8000.th epoch Loss:0.017478225684652512
9000.th epoch Loss:0.017359418203780588
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       100
           1       1.00      0.99      0.99       100
           2       1.00      0.99      0.99       100

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300



In [19]:
X,y=spiral_data_gen(False)
X=X.T
D,N=X.shape
K=len(np.unique(y))

hidden_size=10

model=ComputationalGraph()
model.add((hidden_size,D),ReluGate())
model.add((hidden_size,hidden_size),ReluGate())
model.add((K,hidden_size),SoftmaxGate())
num_epoch=10_000
mode=num_epoch//10


for epoch in range(num_epoch):
    preds=model.forward(X)
    
    if epoch%mode==0:
        loss = (-np.log(preds[y,range(N)])).mean()# mean corect_logprobs
        print('{0}.th epoch Loss:{1}'.format(epoch,loss))
    model.backward(y)
    
    
y_head = np.argmax(model.forward(X), axis=0)
print(classification_report(y, y_head))

0.th epoch Loss:1.0986120099501044
1000.th epoch Loss:1.097885759337879
2000.th epoch Loss:0.4622809226917157
3000.th epoch Loss:0.09837451180661645
4000.th epoch Loss:0.03345261940724648
5000.th epoch Loss:0.023814132218072825
6000.th epoch Loss:0.02014862473904211
7000.th epoch Loss:0.01846945397651559
8000.th epoch Loss:0.017486040516799917
9000.th epoch Loss:0.01672988052676375
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       100
           1       1.00      0.99      0.99       100
           2       0.98      1.00      0.99       100

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300



In [20]:
X,y=spiral_data_gen(False)
X=X.T
D,N=X.shape
K=len(np.unique(y))

hidden_size=2

model=ComputationalGraph()
model.add((hidden_size,D),ReluGate())
model.add((hidden_size,hidden_size),ReluGate())
model.add((K,hidden_size),SoftmaxGate())
num_epoch=10_000
mode=num_epoch//10


for epoch in range(num_epoch):
    preds=model.forward(X)
    
    if epoch%mode==0:
        loss = (-np.log(preds[y,range(N)])).mean()# mean corect_logprobs
        print('{0}.th epoch Loss:{1}'.format(epoch,loss))
    model.backward(y)
    
    
y_head = np.argmax(model.forward(X), axis=0)
print(classification_report(y, y_head))

0.th epoch Loss:1.0986123239301477
1000.th epoch Loss:1.0986122377292749
2000.th epoch Loss:1.0986121278608665
3000.th epoch Loss:1.0986114522096948
4000.th epoch Loss:1.0985913032777168
5000.th epoch Loss:0.7958225122948998
6000.th epoch Loss:0.6694200444507132
7000.th epoch Loss:0.6682061308835694
8000.th epoch Loss:0.6668168552679932
9000.th epoch Loss:0.6652112322770667
              precision    recall  f1-score   support

           0       0.31      0.24      0.27       100
           1       0.71      0.70      0.70       100
           2       0.60      0.75      0.67       100

    accuracy                           0.56       300
   macro avg       0.54      0.56      0.55       300
weighted avg       0.54      0.56      0.55       300



In [ ]:
# Generate an NN API
# Forward() and Backward()
# Layers()
# Implement ReLU, Sigmoid, TanH, Softmax
# IT should be possible to use anyone them in anywhere of the NN.

# Implement ADAM etc.

In [ ]:

from functions import sigmoid,softmax,dsigmoid
from sklearn.metrics import classification_report
from sklearn.datasets import *

# Staged Computation and Matrix-Matrix Multiply Gradient

In [ ]:

# forward pass
W = np.random.randn(5, 10)
X = np.random.randn(10, 3)
D = W.dot(X)

# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) #.T gives the transpose of the matrix
dX = W.T.dot(dD)

In [ ]:
def cross_entropy(p,q):
    """Vectorized Cross entropy - p index of true class
    q true and predicted dist."""
    assert np.all(np.sum(p,axis=1)==1)
    assert np.all(np.sum(q,axis=1)==1)
    return -np.nansum(p*log2(q),axis=1)